<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/energyopt/ThermalEnergyAndNeqSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to TESPy and NeqSim interacations
We start by installing the TESTpy and neqsim python packages.

See: https://tespy.readthedocs.io/en/main/first_steps.html


In [2]:
!pip install tespy
!pip install neqsim

     |████████████████████████████████| 221 kB 14.7 MB/s 
     |████████████████████████████████| 4.2 MB 54.8 MB/s 
     |████████████████████████████████| 31.9 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 49.7 MB/s 


In [3]:
from tespy.networks import Network

# create a network object with water as fluid
fluid_list = ['water']
my_plant = Network(fluids=fluid_list)